In [1]:
import sys
sys.path.append('../build')   # assume an out-of-source build
import exafmm_laplace as fmm
import numpy as np

In [2]:
fmm.__doc__

"exafmm's pybind11 module for Laplace kernel"

#### create sources and targets

In [3]:
nsrcs = 100000
ntrgs = 100000

In [4]:
src_coords = np.random.random((nsrcs, 3))
src_charges = np.random.random(nsrcs)
trg_coords = np.random.random((ntrgs, 3))

In [5]:
sources = fmm.init_sources(src_coords, src_charges)
targets = fmm.init_targets(trg_coords)

#### set FMM parameters

In [6]:
p = 10       # expansion order
ncrit = 100  # max number of bodies allow per leaf box
max_level = 4
fmm.configure(p, ncrit, max_level)

#### build tree

In [7]:
nodes = fmm.build_tree(sources, targets)

#### build interaction lists

In [8]:
skip_P2P = True   # flag to skip near-field evaluation
nodes = fmm.build_list(skip_P2P)

#### check colleagues of a node (after creating the interaction list)

In [9]:
print(len(nodes))
print(type(nodes[500].colleagues))   # list of nodes

4681
<class 'list'>


In [10]:
print(nodes[500].x)

for i, colleague in enumerate(nodes[500].colleagues):
    print(i, colleague.x, colleague.level, colleague.key)

[0.218752, 0.343746, 0.406244]
0 [0.156252, 0.281246, 0.343744] 4 810
1 [0.218752, 0.281246, 0.343744] 4 814
2 [0.281252, 0.281246, 0.343744] 4 1034
3 [0.156252, 0.343746, 0.343744] 4 812
4 [0.218752, 0.343746, 0.343744] 4 816
5 [0.281252, 0.343746, 0.343744] 4 1036
6 [0.156252, 0.406246, 0.343744] 4 826
7 [0.218752, 0.406246, 0.343744] 4 830
8 [0.281252, 0.406246, 0.343744] 4 1050
9 [0.156252, 0.281246, 0.406244] 4 817
10 [0.218752, 0.281246, 0.406244] 4 821
11 [0.281252, 0.281246, 0.406244] 4 1041
12 [0.156252, 0.343746, 0.406244] 4 819
13 [0.218752, 0.343746, 0.406244] 4 823
14 [0.281252, 0.343746, 0.406244] 4 1043
15 [0.156252, 0.406246, 0.406244] 4 833
16 [0.218752, 0.406246, 0.406244] 4 837
17 [0.281252, 0.406246, 0.406244] 4 1057
18 [0.156252, 0.281246, 0.468744] 4 818
19 [0.218752, 0.281246, 0.468744] 4 822
20 [0.281252, 0.281246, 0.468744] 4 1042
21 [0.156252, 0.343746, 0.468744] 4 820
22 [0.218752, 0.343746, 0.468744] 4 824
23 [0.281252, 0.343746, 0.468744] 4 1044
24 [0.15625

Some nodes may not have all 27 colleagues. In that case, `len(colleagues)` is still 27, but the missing colleagues are filled with `None`. For example, the level 1 node `nodes[2]` only have 8 colleagues, other entries are `None`:

In [11]:
print(nodes[2].colleagues)

[None, None, None, None, None, None, None, None, None, None, None, None, <exafmm_laplace.Node object at 0x7f9a12aadb20>, <exafmm_laplace.Node object at 0x7f9a12aadb90>, None, <exafmm_laplace.Node object at 0x7f9a12aadbc8>, <exafmm_laplace.Node object at 0x7f9a12aadc00>, None, None, None, None, <exafmm_laplace.Node object at 0x7f9a12aadc38>, <exafmm_laplace.Node object at 0x7f9a12aadc70>, None, <exafmm_laplace.Node object at 0x7f9a12aadca8>, <exafmm_laplace.Node object at 0x7f9a12aadce0>, None]


#### precompute

In [12]:
fmm.precompute()

#### evaluate potentials

In [13]:
potentials = fmm.evaluate()

P2M                  : 1.1992900e-01
M2M                  : 8.9666000e-02
P2L                  : 1.1230000e-03
M2P                  : 3.2610000e-03
P2P                  : 3.2980000e-03
M2L                  : 6.7501800e-01
L2L                  : 8.4028000e-02
L2P                  : 1.3187800e-01


In [14]:
potentials[::10000]

array([7390.91265992, 8747.74676029, 6821.76158095, 6666.83902003,
       6373.14382885, 7706.89393828, 7002.19983513, 7117.3452941 ,
       6999.21678439, 6850.85003393])

#### check accuracy

For now, this is only valid when skip_P2P is disabled.

In [15]:
fmm.check_accuracy()

-------------- Error ---------------
Potential Error      : 3.6880444e-02
Gradient Error       : 2.9324208e-01


#### update charges and run FMM iteratively

In [ ]:
niters = 10
for i in range(niters):
    print('-'*10 + ' iteration {} '.format(i) + '-'*10)  # print divider between iterations
    src_charges = np.random.random(nsrcs)   # generate new random charges
    fmm.update(src_charges)      # update charges
    fmm.clear()                  # clear values
    potentials = fmm.evaluate()  # evaluate potentials
    fmm.check_accuracy()         # check accuracy